# When Does TAI Date Rollover at LSST?

This notebook calculates the local sunset and nautical twilight times in Chile at the LSST location for a year. It then determines if the TAI rollover happens before each of those times.

In [1]:
from datetime import datetime, timedelta

from astropy.time import Time
from ephem import Observer, Sun
import numpy as np
import pandas as pd
import pytz

In [2]:
lsst_latitude = "-30:14:38.4"
lsst_longitude = "-70:44:57.84"
lsst_height = 2650.0  # meters
lsst_pressure = 750.0  # millibars

In [3]:
lsst_sunset = Observer()
lsst_sunset.long = lsst_longitude
lsst_sunset.lat = lsst_latitude
lsst_sunset.elevation = lsst_height
lsst_sunset.horizon = "-0:34"
lsst_sunset.pressure = lsst_pressure

In [4]:
lsst_nauttwi = Observer()
lsst_nauttwi.long = lsst_longitude
lsst_nauttwi.lat = lsst_latitude
lsst_nauttwi.elevation = lsst_height
lsst_nauttwi.horizon = "-12"
lsst_nauttwi.pressure = lsst_pressure

In [5]:
chile_tz = pytz.timezone('Chile/Continental')
sun = Sun()
full_format = "%Y/%m/%d %H:%M"

In [6]:
def find_rollover(local, tai):
    # Compare the local date to the TAI date. If any of the TAI values are 
    # greater than the local values, the rollover has occurred.
    return tai.year > local.year or tai.month > local.month or tai.day > local.day

In [7]:
original_date = datetime(2018, 1, 1)
end_date = datetime(2019, 1, 1)
days = (end_date - original_date).days
date = original_date

column_names = ["Local Sunset", "TAI Rollover Before Sunset", "Local Nautical Twilight",
                "TAI Rollover Before Nautical Twilight"]
df_rows = []

for i in range(days):
    local_time = chile_tz.localize(date)
    atime = Time(local_time, scale='utc')
    dtd = atime.datetime
    lsst_sunset.date = dtd
    lsst_nauttwi.date = dtd
    sunset = lsst_sunset.next_setting(sun).datetime()
    naut_twi = lsst_nauttwi.next_setting(sun).datetime()
    sunset_local = chile_tz.fromutc(sunset)
    nauttwi_local = chile_tz.fromutc(naut_twi)
    sunset_tai = Time(sunset, scale='utc').tai.datetime
    nauttwi_tai = Time(naut_twi, scale='utc').tai.datetime
    rollover_sunset = find_rollover(sunset_local, sunset_tai)
    rollover_nauttwi = find_rollover(nauttwi_local, nauttwi_tai)
    df_rows.append(pd.DataFrame([[sunset_local.strftime(full_format),
                                 rollover_sunset,
                                 nauttwi_local.strftime(full_format),
                                 rollover_nauttwi]], columns=column_names))
    date += timedelta(1)
df = pd.concat(df_rows, ignore_index=True)

In [8]:
# Find the number of days the rollover happens before sunset
days_before_sunset = df.loc[df[column_names[1]] == True]
print(days_before_sunset.shape[0])

0


In [9]:
# Find the rows where the rollover happens before nautical twilight
rows_before = np.where(df[column_names[3]] == True)
# Find the rows where the rollover happens after nautical twilight
rows_after = np.where(df[column_names[3]] == False)
print("Number of Days Rollover Occurs Before Nautical Twilight: {}".format(len(rows_before[0])))
print("Number of Days Rollover Occurs After Nautical Twilight: {}".format(len(rows_after[0])))

Number of Days Rollover Occurs Before Nautical Twilight: 133
Number of Days Rollover Occurs After Nautical Twilight: 232


In [10]:
print("Rollover After Nautical Twilight Starts: {}".format(df.iloc[rows_after[0][0]][0].split()[0]))
print("Rollover After Nautical Twilight Ends: {}".format(df.iloc[rows_after[0][-1]][0].split()[0]))

Rollover After Nautical Twilight Starts: 2018/03/11
Rollover After Nautical Twilight Ends: 2018/10/28
